In [1]:
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


gm_per_lb = 453.59237
l_per_gal = 3.785411784
cm_per_in = 2.54
cm_per_ft = 12 * cm_per_in
ml_per_gal = 1e3 * l_per_gal
min_per_day = 24 * 60                           ## 1440 minutes per day
gpmPft2_to_cmPs = 6.79e-2                       ## 6.79e-2 cm/s per gpm/ft**2



In [2]:
pfas_prop = pd.read_excel('../../PSDM/PFAS_properties.xlsx', index_col=0, header=0)

mw_anions = {'CHLORIDE': 35.45, 'SULFATE': 96.06, 
            'BICARBONATE': 12.01, ## as carbon
            'NITRATE': 14, ## as nitrogen
            }

In [3]:
## Use for running IX without saving a file
from ixpy import hsdmix, psdmix
from io import BytesIO

def io_data(data_dict):
    '''
    Parameters
    ----------
    data_dict : dictionary of dataframes
        must be in the form:
            
            {'Cin': cin_df,
             'params': params_df,
             'ions': ions_df
             }


    Returns
    -------
    iex_inmem : BytesIO RAM file
        a RAM file that can be supplied to HSDMIX in place of saved file

    '''
    
    iex_inmem=BytesIO()
    # writer=pd.ExcelWriter(iex_inmem, engine='xlsxwriter')
    # for sheet_name in data_dict.keys():
    #     data_dict[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)
    # writer.save()
    # iex_inmem.seek(0,0)
    with pd.ExcelWriter(iex_inmem) as writer:
        for sheet_name in data_dict.keys():
            data_dict[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)
    iex_inmem.seek(0,0)
    return iex_inmem

## Test with individual mass transfer coefficients

In [5]:
fn = './data/test_input_multi_mass_transfer.xlsx'

IXcol = hsdmix.HSDMIX(fn)
t, u = IXcol.solve()



df = hsdmix.conv_iex_u(u, t, IXcol.ions)
print(df)

KeyError: 'q'

In [ ]:
for comp in df.columns:
    if comp not in ['CHLORIDE', 'SULFATE', 'BICARBONATE', 'NITRATE']:
        plt.plot(df.index, 
                 df[comp].values,
                 label=comp)

plt.xlabel('Time (days)')
plt.ylabel('Concentration (ng/L)')
plt.legend(loc='upper left')

## Test with global mass transfer
Past compatibility

In [ ]:
fn = './data/reg_test_input.xlsx'

IXcol = hsdmix.HSDMIX(fn)
# print(IXcol.__dict__)  ## shows kL, Ds in the ions data
t, u = IXcol.solve()



df = hsdmix.conv_iex_u(u, t, IXcol.ions)
print(df)

for comp in df.columns:
    if comp in ['CHLORIDE', 'SULFATE', 'BICARBONATE', 'NITRATE']:
        plt.plot(df.index, 
                 df[comp].values,
                 label=comp)

plt.xlabel('Time (days)')
plt.ylabel('Concentration (ng/L)')
plt.legend(loc='upper left')